In [12]:
!pip install fuzzywuzzy python-Levenshtein openpyxl groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.7 MB/s eta 0:00:00


In [47]:
import pandas as pd
import re

# Load the dataset
path_report= pd.read_csv("/content/Pathology report.csv")
path_report

,Title,Report
0,Acute Respiratory Disease Syndrome in Horse,On 22 September 1994 an emergency situation wa...
1,Idiopathic Heart Failure/Liver Necrosis Syndro...,Grower pigs on a Beaudesert piggery began suff...
2,Avian pulmonary tuberculosis in broiler breeders,Avian pulmonary tuberculosis was diagnosed his...
3,Tail Rot in Eels,Heavy losses occurred in eels (Anguilla reinha...
4,Crotalaria dissitiflora (pyrrolizidine alkaloi...,Typical pyrrolizidine alkaloidosis was seen in...
5,Nitrate-nitrite poisoning from button grass in...,Nitrate-nitrite poisoning was diagnosed in a g...
6,Oleander poisoning in cattle,Poisoning by oleander (Nerium oleander) was di...
7,Cardiomyopathy and Atrial Thrombosis in a Foal,Clinical examination of a 6-months old thoroug...
8,Herpes virus pneumonia in a feedlot steer,Portions of lung from a feedlot steer with lab...
9,Dermatophyte infection as a cause of Pyogranul...,Microsporum canis was cultured from chronic su...


**ENTITY EXTRACTION USING LLAMA3**

Extract the following entities from the pathology report: Animal Species,Symptoms, Necropsy Findings, Diagnoses,Etiological Agents




In [49]:
import groq
GROQ_API_KEY="" #you can get an API KEY HERE(https://console.groq.com/keys)
groq_client=groq.Groq(api_key=GROQ_API_KEY)


def extract_entities(report):
    sys_prompt = "You are a veterinary pathology expert in extracting entities from pathology reports."
    question = f"""
    Extract the following entities from the pathology report:
    1. Animal Species
    2. Symptoms
    3. Necropsy Findings
    4. Diagnoses
    5. Etiological Agents

    Pathology Report:
    {report}

    Output:
    """
    try:
        response = groq_client.chat.completions.create(
            model='llama-3.3-70b-versatile',
            response_format={'type': 'text'},
            messages=[
                {'role': 'system', 'content': sys_prompt},
                {'role': 'user', 'content': question}
            ],
            temperature=0.1,
            top_p=0.9,  # Nucleus sampling
            seed=79
        )
        output = response.choices[0].message.content
        return output
    except Exception as e:
        return f"Error: {str(e)}"

# Load CSV File
input_file = "/content/Pathology report.csv"  # Replace with your file path
output_file = "extracted_entities.csv"

# Read the Input CSV File
data = pd.read_csv(input_file)

# Ensure Columns Exist
if 'Title' not in data.columns or 'Report' not in data.columns:
    raise ValueError("The CSV file must have 'Title' and 'Report' columns.")

# Initialize Lists for New Columns
data["Animal Species"] = ""
data["Symptoms"] = ""
data["Necropsy Findings"] = ""
data["Diagnoses"] = ""
data["Etiological Agents"] = ""

# Process Each Report
for index, row in data.iterrows():
    report = row["Report"]
    output = extract_entities(report)

    # Debugging: Print the raw API response
    print(f"Raw response for row {index}:\n{output}\n")

    # Parse Output into Columns
    try:
        if not output or "Error" in output:
            raise ValueError(f"Invalid output: {output}")

        # Flexible parsing based on response content
        parsed_output = output.split("\n")
        for line in parsed_output:
            if "Animal Species" in line:
                data.at[index, "Animal Species"] = line.split(":")[1].strip()
            elif "Symptoms" in line:
                data.at[index, "Symptoms"] = line.split(":")[1].strip()
            elif "Necropsy Findings" in line:
                data.at[index, "Necropsy Findings"] = line.split(":")[1].strip()
            elif "Diagnoses" in line:
                data.at[index, "Diagnoses"] = line.split(":")[1].strip()
            elif "Etiological Agents" in line:
                data.at[index, "Etiological Agents"] = line.split(":")[1].strip()
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        continue

# Save the Result to a New CSV File
data.to_csv(output_file, index=False)

Raw response for row 0:
Here are the extracted entities from the pathology report:

1. **Animal Species**: Horses (Equine)
2. **Symptoms**: 
   - Depression
   - Anorexia
   - Weakness
   - High fever (40-41°C)
   - Severe respiratory difficulty
   - Frothy discharge from the nose
   - Jaundice (in some animals)
3. **Necropsy Findings**:
   - Cyanosis and petechiation of the buccal mucosa
   - Stable froth in the trachea and bronchi
   - Severe congestion and oedema of the lungs
   - Multiple focal haemorrhages on the pleural and cut surfaces of the lungs
   - Petechiation of the peritoneum and epicardial haemorrhage (in some animals)
   - Petechiation of the gastric mucosa (in one horse)
4. **Diagnoses**: 
   - Peracute or acute interstitial pneumonia
   - Intravascular coagulation in arterioles and alveolar capillaries
5. **Etiological Agents**: 
   - A paramyxovirus (later identified as a morbillivirus, related to the measles, distemper, and rinderpest group, but with no relationshi

In [1]:
def extract_entities(text):
    entities = {
        "Animal Species": "",
        "Symptoms": "",
        "Necropsy Findings": "",
        "Diagnoses": "",
        "Etiological Agents": ""
    }

    patterns = {
        "Animal Species": r"1\. \*\*Animal Species\*\*:(.+?)(?=2\. \*\*Symptoms)",
        "Symptoms": r"2\. \*\*Symptoms\*\*:(.+?)(?=3\. \*\*Necropsy Findings)",
        "Necropsy Findings": r"3\. \*\*Necropsy Findings\*\*:(.+?)(?=4\. \*\*Diagnoses)",
        "Diagnoses": r"4\. \*\*Diagnoses\*\*:(.+?)(?=5\. \*\*Etiological Agents)",
        "Etiological Agents": r"5\. \*\*Etiological Agents\*\*:(.+?)(?=\n\n|$)"
    }

    for entity, pattern in patterns.items():
        match = re.search(pattern, text, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            extracted_text = re.sub(r'^\s*[-*+]\s*', '', extracted_text, flags=re.MULTILINE)
            extracted_text = re.sub(r'\*\*', '', extracted_text)
            extracted_text = re.sub(r'\s+', ' ', extracted_text)
            entities[entity] = extracted_text

    return entities

def parse_pathology_reports(raw_text):
    report_pattern = r"Raw response for row (\d+):(.*?)(?=Raw response for row \d+:|$)"
    reports = re.findall(report_pattern, raw_text, re.DOTALL)

    processed_reports = []
    for index, report_text in reports:
        entities = extract_entities(report_text)
        entities["Report ID"] = int(index)
        processed_reports.append(entities)

    return processed_reports

def create_dataframe(processed_reports):
    df = pd.DataFrame(processed_reports)
    columns = ["Report ID"] + [col for col in df.columns if col != "Report ID"]
    df = df[columns]
    df = df.sort_values("Report ID")

    return df

def main(raw_text):
    processed_reports = parse_pathology_reports(raw_text)

    df = create_dataframe(processed_reports)

    print(f"Processed {len(df)} pathology reports.")

    df.to_csv("pathology_reports.csv", index=False)
    print("Data saved to pathology_reports.csv")
    return df

In [7]:
if __name__ == "__main__":
    with open("/content/pathology_reports.txt", "r") as f:
        raw_text = f.read()

    # Process the data
    df = main(raw_text)
    df.head(5)

Processed 47 pathology reports.
Data saved to pathology_reports.csv


In [8]:
df

,Report ID,Animal Species,Symptoms,Necropsy Findings,Diagnoses,Etiological Agents
0,0,Horses (Equine),Depression Anorexia Weakness High fever (40-41...,Cyanosis and petechiation of the buccal mucosa...,Peracute or acute interstitial pneumonia Intra...,A paramyxovirus (later identified as a morbill...
1,1,"Pigs (specifically, grower pigs and weaners)",Dyspnoea (difficulty breathing) Sudden death N...,Dilation of the heart Excessive volumes of flu...,The cause of the syndrome is presently unresol...,Nutritionally dense diet high in energy and pr...
2,2,Broiler breeders (chickens),Anorexia Lethargy Loss of weight Diarrhoea,Multiple small caseous granulomas in lung tiss...,Avian pulmonary tuberculosis,Mycobacterium avium serotypes 1 or 2 (typical ...
3,3,Eels (Anguilla reinhardti),Heavy losses Severe tail erosion Extensive ski...,Severe tail erosion with extensive skin ulcera...,Tail rot,Fungal agent: Saprolegnia sp Bacterial agent: ...
4,4,Horses,"Not explicitly mentioned, but it can be inferr...",Typical pyrrolizidine alkaloidosis lesions wer...,Pyrrolizidine alkaloidosis,"Crotalaria dissitiflora (grey rattlepod), spec..."
5,5,Brahman steers (cattle),Found dead Sick Died within a few hours,Dark brown discolouration of the blood,Nitrate-nitrite poisoning,Nitrate (NO3) Nitrite (NO2) Button grass (Dact...
6,6,Santa Gertrudis-cross weaners (cattle),"Severe dysentery, death (in 5 animals)",Haemorrhage and congestion in the large intest...,Poisoning by oleander (Nerium oleander),Oleander (Nerium oleander)
7,7,Thoroughbred foal (equine),Loss of condition Depression Recumbency Exagge...,Hydropericardium Multifocal areas of myocardia...,Cardiomyopathy Atrial thrombosis,Congenital cardiomyopathy Acquired condition S...
8,8,"Bovine (specifically, a feedlot steer)",Laboured breathing,Extensive grey consolidation with pale broncho...,Infectious Bovine Rhinotracheitis (IBR) infection,"Bovine herpesvirus (specifically, the virus th..."
9,9,Cat,Chronic suppurative cutaneous lesions in the l...,"Severe deep dermal inflammatory reaction, mult...","Cutaneous mycobacteriosis (provisional), Mycetoma","Microsporum canis (fungus), non-pigmented yeas..."


**Mapping of entities to SNOMED CODE**

In [13]:
import re
import pandas as pd
import json
from fuzzywuzzy import fuzz, process

class ClinicalTerminologyMapper:
    def __init__(self):
        self.snomed_diagnoses = {
            "interstitial pneumonia": "36485005",
            "intravascular coagulation": "67406007",
            "cardiomyopathy": "85898001",
            "tuberculosis": "371569005",
            "avian pulmonary tuberculosis": "232156003",
            "mycobacteriosis": "42765004",
            "hendra virus infection": "442439002",
            "tail rot": "127432006",
            "pyrrolizidine alkaloidosis": "71533009",
            "nitrate poisoning": "21879003",
            "oleander poisoning": "82322001",
            "infectious bovine rhinotracheitis": "10608251000119103",
            "nodular dermatofibrosis": "238798009",
            "infectious laryngotracheitis": "44396000",
            "malignant catarrhal fever": "65092000",
            "toxoplasmosis": "58763007",
            "coccidiosis": "56457000",
            "botryomycosis": "81283009",
            "cholera": "63650001",
            "rhabdomyosarcoma": "443331004"
        }

        self.snomed_species = {
            "horse": "35354009",
            "pig": "448771007",
            "chicken": "387961004",
            "eel": "90704004",
            "turkey": "47352007",
            "sheep": "125097000",
            "cattle": "34618005",
            "goat": "448169003",
            "dog": "448771007",
            "cat": "388606001",
            "rabbit": "74892005",
            "koala": "81863003",
            "kangaroo": "52518008",
            "wombat": "83037005",
            "crocodile": "25295006",
            "ostrich": "426812001",
            "mouse": "387964007"
        }

        self.snomed_symptoms = {
            "fever": "386661006",
            "anorexia": "79890006",
            "weakness": "13791008",
            "depression": "35489007",
            "respiratory difficulty": "267036007",
            "nasal discharge": "64531003",
            "jaundice": "18165001",
            "dyspnoea": "267036007",
            "sudden death": "27942005",
            "diarrhoea": "62315008",
            "lethargy": "84229001",
            "weight loss": "89362005",
            "alopecia": "56317004",
            "pyrexia": "386661006",
            "dysentery": "236071009",
            "bloat": "83147009",
            "ataxia": "9650009",
            "incoordination": "302348007"
        }

        self.snomed_pathogens = {
            "paramyxovirus": "49872002",
            "morbillivirus": "60780000",
            "hendra virus": "81167003",
            "mycobacterium avium": "29686008",
            "saprolegnia": "252734000",
            "aeromonas hydrophila": "41381008",
            "pasteurella multocida": "52584002",
            "staphylococcus aureus": "3092008",
            "toxoplasma gondii": "240815006",
            "bovine herpesvirus": "54382003",
            "chlamydia": "45261009",
            "aspergillus": "59953007"
        }



    def find_best_match(self, text, dictionary, threshold=70):
        if not text:
            return None, None

        # Search for exact matches first
        for term in dictionary.keys():
            if term.lower() in text.lower():
                return term, dictionary[term]

        # If no exact match, try fuzzy matching
        best_match, score = process.extractOne(text, dictionary.keys())
        if score >= threshold:
            return best_match, dictionary[best_match]

        return None, None

    def map_diagnosis(self, diagnosis_text):
        snomed_term, snomed_code = self.find_best_match(diagnosis_text, self.snomed_diagnoses)


        return {
            "SNOMED_diagnosis_term": snomed_term,
            "SNOMED_diagnosis_code": snomed_code

        }

    def map_species(self, species_text):
        species_term, species_code = self.find_best_match(species_text, self.snomed_species)

        return {
            "SNOMED_species_term": species_term,
            "SNOMED_species_code": species_code
        }

    def map_symptoms(self, symptoms_text):
        mapped_symptoms = []
        symptom_list = re.split(r'[,;.]', symptoms_text)
        for symptom in symptom_list:
            symptom = symptom.strip()
            if symptom:
                term, code = self.find_best_match(symptom, self.snomed_symptoms)
                if term:
                    mapped_symptoms.append({"term": term, "code": code})

        return mapped_symptoms

    def map_pathogens(self, pathogens_text):
        mapped_pathogens = []

        pathogen_list = re.split(r'[,;.]', pathogens_text)
        for pathogen in pathogen_list:
            pathogen = pathogen.strip()
            if pathogen:
                term, code = self.find_best_match(pathogen, self.snomed_pathogens)
                if term:
                    mapped_pathogens.append({"term": term, "code": code})

        return mapped_pathogens

def extract_entities(text):

    entities = {
        "Animal Species": "",
        "Symptoms": "",
        "Necropsy Findings": "",
        "Diagnoses": "",
        "Etiological Agents": ""
    }

    patterns = {
        "Animal Species": r"1\. \*\*Animal Species\*\*:(.+?)(?=2\. \*\*Symptoms)",
        "Symptoms": r"2\. \*\*Symptoms\*\*:(.+?)(?=3\. \*\*Necropsy Findings)",
        "Necropsy Findings": r"3\. \*\*Necropsy Findings\*\*:(.+?)(?=4\. \*\*Diagnoses)",
        "Diagnoses": r"4\. \*\*Diagnoses\*\*:(.+?)(?=5\. \*\*Etiological Agents)",
        "Etiological Agents": r"5\. \*\*Etiological Agents\*\*:(.+?)(?=\n\n|$)"
    }

    for entity, pattern in patterns.items():
        match = re.search(pattern, text, re.DOTALL)
        if match:
            extracted_text = match.group(1).strip()
            extracted_text = re.sub(r'^\s*[-*+]\s*', '', extracted_text, flags=re.MULTILINE)
            extracted_text = re.sub(r'\*\*', '', extracted_text)
            extracted_text = re.sub(r'\s+', ' ', extracted_text)
            entities[entity] = extracted_text

    return entities

def parse_pathology_reports(raw_text):
    report_pattern = r"Raw response for row (\d+):(.*?)(?=Raw response for row \d+:|$)"
    reports = re.findall(report_pattern, raw_text, re.DOTALL)

    processed_reports = []
    for index, report_text in reports:
        entities = extract_entities(report_text)
        entities["Report ID"] = int(index)
        processed_reports.append(entities)

    return processed_reports

def map_to_clinical_standards(df):
    mapper = ClinicalTerminologyMapper()

    df["SNOMED_species"] = None
    df["SNOMED_diagnosis"] = None
    df["SNOMED_symptoms"] = None
    df["SNOMED_pathogens"] = None


    for idx, row in df.iterrows():

        species_mapping = mapper.map_species(row["Animal Species"])
        df.at[idx, "SNOMED_species"] = json.dumps(species_mapping)

        diagnosis_mapping = mapper.map_diagnosis(row["Diagnoses"])
        df.at[idx, "SNOMED_diagnosis"] = json.dumps(diagnosis_mapping)

        symptoms_mapping = mapper.map_symptoms(row["Symptoms"])
        df.at[idx, "SNOMED_symptoms"] = json.dumps(symptoms_mapping)

        pathogens_mapping = mapper.map_pathogens(row["Etiological Agents"])
        df.at[idx, "SNOMED_pathogens"] = json.dumps(pathogens_mapping)

    return df

def main(raw_text):
    processed_reports = parse_pathology_reports(raw_text)
    df = pd.DataFrame(processed_reports)
    columns = ["Report ID"] + [col for col in df.columns if col != "Report ID"]
    df = df[columns]
    df = df.sort_values("Report ID")
    df = map_to_clinical_standards(df)

    print(f"Processed {len(df)} pathology reports with clinical standard mappings.")
    df.to_csv("pathology_reports_mapped.csv", index=False)

    return df


In [14]:
if __name__ == "__main__":
    with open("pathology_reports.txt", "r") as f:
        raw_text = f.read()
    df = main(raw_text)
    df.head()

Processed 47 pathology reports with clinical standard mappings.


In [15]:
df

,Report ID,Animal Species,Symptoms,Necropsy Findings,Diagnoses,Etiological Agents,SNOMED_species,SNOMED_diagnosis,SNOMED_symptoms,SNOMED_pathogens
0,0,Horses (Equine),Depression Anorexia Weakness High fever (40-41...,Cyanosis and petechiation of the buccal mucosa...,Peracute or acute interstitial pneumonia Intra...,A paramyxovirus (later identified as a morbill...,"{""SNOMED_species_term"": ""horse"", ""SNOMED_speci...","{""SNOMED_diagnosis_term"": ""interstitial pneumo...","[{""term"": ""fever"", ""code"": ""386661006""}]","[{""term"": ""paramyxovirus"", ""code"": ""49872002""}..."
1,1,"Pigs (specifically, grower pigs and weaners)",Dyspnoea (difficulty breathing) Sudden death N...,Dilation of the heart Excessive volumes of flu...,The cause of the syndrome is presently unresol...,Nutritionally dense diet high in energy and pr...,"{""SNOMED_species_term"": ""pig"", ""SNOMED_species...","{""SNOMED_diagnosis_term"": null, ""SNOMED_diagno...","[{""term"": ""dyspnoea"", ""code"": ""267036007""}]",[]
2,2,Broiler breeders (chickens),Anorexia Lethargy Loss of weight Diarrhoea,Multiple small caseous granulomas in lung tiss...,Avian pulmonary tuberculosis,Mycobacterium avium serotypes 1 or 2 (typical ...,"{""SNOMED_species_term"": ""chicken"", ""SNOMED_spe...","{""SNOMED_diagnosis_term"": ""tuberculosis"", ""SNO...","[{""term"": ""anorexia"", ""code"": ""79890006""}]","[{""term"": ""mycobacterium avium"", ""code"": ""2968..."
3,3,Eels (Anguilla reinhardti),Heavy losses Severe tail erosion Extensive ski...,Severe tail erosion with extensive skin ulcera...,Tail rot,Fungal agent: Saprolegnia sp Bacterial agent: ...,"{""SNOMED_species_term"": ""eel"", ""SNOMED_species...","{""SNOMED_diagnosis_term"": ""tail rot"", ""SNOMED_...",[],"[{""term"": ""saprolegnia"", ""code"": ""252734000""}]"
4,4,Horses,"Not explicitly mentioned, but it can be inferr...",Typical pyrrolizidine alkaloidosis lesions wer...,Pyrrolizidine alkaloidosis,"Crotalaria dissitiflora (grey rattlepod), spec...","{""SNOMED_species_term"": ""horse"", ""SNOMED_speci...","{""SNOMED_diagnosis_term"": ""pyrrolizidine alkal...","[{""term"": ""fever"", ""code"": ""386661006""}]",[]
5,5,Brahman steers (cattle),Found dead Sick Died within a few hours,Dark brown discolouration of the blood,Nitrate-nitrite poisoning,Nitrate (NO3) Nitrite (NO2) Button grass (Dact...,"{""SNOMED_species_term"": ""cattle"", ""SNOMED_spec...","{""SNOMED_diagnosis_term"": ""nitrate poisoning"",...",[],[]
6,6,Santa Gertrudis-cross weaners (cattle),"Severe dysentery, death (in 5 animals)",Haemorrhage and congestion in the large intest...,Poisoning by oleander (Nerium oleander),Oleander (Nerium oleander),"{""SNOMED_species_term"": ""cattle"", ""SNOMED_spec...","{""SNOMED_diagnosis_term"": ""nitrate poisoning"",...","[{""term"": ""dysentery"", ""code"": ""236071009""}, {...",[]
7,7,Thoroughbred foal (equine),Loss of condition Depression Recumbency Exagge...,Hydropericardium Multifocal areas of myocardia...,Cardiomyopathy Atrial thrombosis,Congenital cardiomyopathy Acquired condition S...,"{""SNOMED_species_term"": null, ""SNOMED_species_...","{""SNOMED_diagnosis_term"": ""cardiomyopathy"", ""S...","[{""term"": ""depression"", ""code"": ""35489007""}]",[]
8,8,"Bovine (specifically, a feedlot steer)",Laboured breathing,Extensive grey consolidation with pale broncho...,Infectious Bovine Rhinotracheitis (IBR) infection,"Bovine herpesvirus (specifically, the virus th...","{""SNOMED_species_term"": null, ""SNOMED_species_...","{""SNOMED_diagnosis_term"": ""infectious bovine r...",[],"[{""term"": ""bovine herpesvirus"", ""code"": ""54382..."
9,9,Cat,Chronic suppurative cutaneous lesions in the l...,"Severe deep dermal inflammatory reaction, mult...","Cutaneous mycobacteriosis (provisional), Mycetoma","Microsporum canis (fungus), non-pigmented yeas...","{""SNOMED_species_term"": ""cat"", ""SNOMED_species...","{""SNOMED_diagnosis_term"": ""mycobacteriosis"", ""...",[],[]
